# Understanding Hooks 

[reference] <br>
https://www.digitalocean.com/community/tutorials/pytorch-hooks-gradient-clipping-debugging



PyTorch Hooks는 모델 아키텍처를 변경하지 않고도 학습 과정을 디버깅, 활성화값(activation)을 시각화하거나 기울기를 수정할 수 있다. <br>
Hooks는 내부 동작이 불투명한 딥러닝 모델의 내부 흐름을 관찰하고 해석하는데 큰 도움이 된다. <br>

## PyTorch Hooks 소개

Hooks가 PyTorch에서 중요한 역할을 하는 이유 중 하나는 역전파(backpropagation) 중에 모델과 상호작용할 수 있도록 해주기 때문에 중요하다. <br>
예를 들어, 학습 중간에 특정 층의 출력값이나 기울기를 기록하거나 조작 할 수 있어 디버깅 및 모델 해석에 매우 효과적이다. <br>
Hook은 단순히 하나의 함수이며, Tensor나 nn.Module에 연결하면 forward나 backward 과정 중에 자동으로 실행된다. <br>
여기서 말하는 forward는 module 클래스 내에 forward() 매서드를 의미하는 것이 아니고 torch.autograd.Function 클래스 내부에서 이루어진다. <br>
(이 부분은 PyTorch가 모든 연산을 자동으로 기록하여 역전파를 자동 계산할 수 있게 해주는 기반 구조이다.)

PyTorch에서 연산을 통해 생성된 모든 Tensor에는 grad_fn이라는 속성이 존재한다. 이 grad_fn는 torch.autograd.Function의 인스턴스로 해당 텐서를 생성한 연산을 나타낸다. 이 덕분에 PyTorch는 어떤 연산이 어떻게 이루어졌는지 추적할 수 있고, 역전파 시 그 경로를 따라 기울기를 계산할 수 있다. <br>
예를 들어 tensor = tensor1 + tensor2 를 계산하면 output인 tensor에는 AddBackward 타입의 grad_fn이 붙는다. 이는 해당 덧셈 연산을 통해 기울기를 어떻게 계산할지를 내부에서 알고 있다는 뜻이다. (이 부분이 이해되지 않는다면 PyTorch 계산 그래프에 관련된 글을 찾아보길 바란다.) <br>
간단히 요약하자면, 연산을 통해 생성된 모든 텐서는 grad_fn을 통해 생성 경로를 추적할 수 있다는 점이다. 

여기서 중요한 사실은 nn.Linear 같은 nn.Module 객체는 내부적으로 여러 연산으로 구성되어 있다. Linear는 $ Y = W * X + B$ 와 같은 방정식으로 행렬 곱셈 후 덧셈 이라는 연산을 수행한다. autograd 수준에서는 곱셈과 덧셈 각각에 대해 별도의 forward 연산이 발생하는 거지, 하나의 forward() 함수 호출마 있는 것은 아니다. <br>
이 점을 고려하지 않고 Hooks를 사용하면 전체 레이어가 아닌 개별 연산에 hook이 걸릴 수 있어, 여러 개의 출력이 생기거나 예상치 못한 동작이 발생할 수 있다. 

## Hooks 종류

1. Forward Hook : 순전파 시 (모델이 입력을 받아 출력을 계산하는 과정) 실행된다. 
2. Backward Hook : 역전파 시 (기울기를 계산해 파라미터를 업데이트하는 과정) 실행된다.

PyTorch는 모든 연산을 자동 미분할 수 있도록, 내부적으로 torch.autograd.Function을 사용해 forward/backward 동작을 추적한다. 이 구조에 hook도 연결되어 동작한다. 

- register_forward_hook() : 순전파 때 작동
- register_full_backward_hook() : 역전파 때 작동
(register_backward_hook()은 권장되지 않고 full을 사용할 것)

## Tensor에 대한 Hook

Hook은 특정한 형식을 갖춘 함수이다. Hook이 실행된다는 것은 실제로는 해당 함수를 PyTorch가 자동으로 호출한다는 뜻이다. 

In [ ]:
# Tensor에 대한 backward hook 함수의 형태는 다음과 같다. 

hook(grad) -> Tensor 또는 None

## 설명
# grad : backward가 호출된 후, 해당 Tensor의 .grad 속성에 저장되는 기울기 값
# 이 함수는 grad를 변경하지 않고 반환하거나, 수정된 새로운 Tensor를 반환해야 한다. 
# 반환된 Tensor는 이후 역전파 계산에 원래의 기울기 대신 사용
# None을 반환하면 기존의 grad 값이 그대로 사용된다. 

### Tensor는 forward hook을 지원하지 않는다. 

즉 forward hook 기능이 존재하지 않고, backward hook만 존재

In [7]:
import torch

a = torch.ones(5)
a.requires_grad = True

b = 2 * a
b.retain_grad() # b는 non-leaf 텐서이므로, .grad를 유지하려면 retain_grad()를 호출해야 한다.

c = b.mean()
c.backward()

print(a.grad,"\n",b.grad)

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) 
 tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [15]:
a = torch.ones(5)
a.requires_grad = True

b = 2 * a
b.retain_grad() 

def print_hook(x):
    print(x)
    return None

b.register_hook(print_hook) # hook 함수 등록할 때는 그냥 함수 자체를 괄호에 넣어야 함
b.mean().backward()

print(a.grad,"\n",b.grad)


tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) 
 tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [16]:
a = torch.ones(5)
a.requires_grad = True

b = 2*a
b.retain_grad()

b.mean().backward() 

print(a.grad, b.grad)

b.grad *= 2

print(a.grad, b.grad)

# b.grad *= 2는 b의 기울기를 변경했지만, a의 기울기는 이미 계산이 끝난 뒤라 바뀌지 않음
# 즉, 역전파 중간에 b.grad를 수정하고 싶다면, hook을 써야 함
# 그렇지 않으면 b에 의존하는 모든 텐서의 grad를 수동으로 수정해야 함

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000])


## nn.Module 객체에 대한 Hook

nn.Module 객체에 대해 hook 함수의 형식(signature)는 다음과 같다. 

In [ ]:
# Backward hook:

hook(module, grad_input, grad_output) -> Tensor 또는 None

# Forward hook:
hook(module, input, output) -> None

## 설명
# module : hook이 등록된 nn.Module 객체 자체
# input/grad_input : 해당 모듈의 입력 값 또는 입력에 대한 기울기
# output / grad_output : 해당 모듈의 출력 값 또는 출력에 대한 기울기
# forward hook은 출력값을 관찰하는 데 유용하고, backward hook은 기울기를 추적하거나 조정하는 데 사용

## 왜 Module 객체에 hook 을 조심히 사용해야 하는가

hook을 사용하기 위해서는 module 내부의 추상화를 깨야 한다.<br>
nn.Module은 일반적으로 하나의 레이어를 나타내는 모듈화된 객체이지만 실제로는 여러 연산을 수행할 수 있다. 즉 하나의 모듈에 대해 여러 번의 forward나 backward가 일어날 수 있으며, 이 구조를 이해하지 못하면 hook의 위치와 의미를 혼동하게 된다. 즉 내부적으로 Add나 MatMul 같은 여러 연산이 묶여 있기 때문에 hook이 어디에 걸렸는지 명확히 인식해야 한다. 

## 예) nn.Linear는 내부적으로 어떻게 구성되어 있을까?

Linear 레이어는 내부적으로 행렬 곱과 덧셈 두가지 연산을 수행한다. $Y = W * X + b$ <br>
이 두 연산은 별도의 노드로 인식한다. 따라서 이 레이어에 forward hook을 등록하면 input이 Tensor가 아닌 **tuple 형태**일 수 있다. -> 이는 각 개별 연산의 입력을 의미하기 때문이다. <br>
마찬가지로, output도 단일 값이 아닌, 연산 시점의 특정 출력값일 수 있다. 

> Hook을 모듈 수준에 걸면 전체 레이어의 입출력이나 기울기를 다루는 게 아니라, <br> 그 시점의 연산 흐름 중 일부를 다룬다. 

In [21]:
from torch import nn

class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 10, 2, stride = 2)
        self.relu = nn.ReLU()
        #self.flatten = nn.Flatten()
        self.flatten = lambda x: x.view(-1)
        self.fc1 = nn.Linear(160, 5)

    def forward(self, x):
        x = self.relu(self.conv(x))
        return self.fc1(self.flatten(x))

def hook_fn(m, i, o):
    print(m)
    print("--------------Input Grad---------------")

    for grad in i:
        try:
            print(grad.shape)
        except AttributeError:
            print("None found for Gradient")
    
    print("--------------Output Grad---------------")
    for grad in o:
        try:
            print(grad.shape)
        except AttributeError:
            print("None found for Gradient")
    print("\n")

net = myNet()

net.conv.register_backward_hook(hook_fn)
net.fc1.register_backward_hook(hook_fn)

inp = torch.randn(1, 3, 8, 8)
out = net(inp)
(1 - out.mean()).backward()

Linear(in_features=160, out_features=5, bias=True)
--------------Input Grad---------------
torch.Size([1, 5])
--------------Output Grad---------------
torch.Size([5])


Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))
--------------Input Grad---------------
None found for Gradient
torch.Size([10, 3, 2, 2])
torch.Size([10])
--------------Output Grad---------------
torch.Size([1, 10, 4, 4])




### Conv2d의 Grad 해석
- grad_input:
    -  [10, 3, 2, 2]: weight의 기울기
    -  [10]: bias의 기울기
    -  None : 입력 feature map의 기울기 (conv 앞 레이어에서 받아야 하므로 이 시점에는 없음)
  Conv는 내부적으로 im2col 같은 방법으로 이미지 데이터를 펼쳐서 행렬곱 방식으로 계산함<br>
    -> 이로 인해 연산이 나뉘고 hook이 예상과 다르게 동작할 수 있음

### Linear의 Grad 해석
- grad_input 이 둘 다 [5] : 이게 왜이럴까?
    - 사실 fc1의 weight는 [160, 5]인데 왜 [5] 일까?
    - 내부 동작을 정확하게 이해하지 않으면 기울기 방향과 구조가 직관과 다를 수 있음